In [ ]:
pip install gql[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 838.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import plotly.express as px
import json

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

## Balancer V2

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
            feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = df11['inputTokens'].map(lambda x:[i['id'] for i in x]).apply(pd.Series)
num_columns12 = len(df12.columns)
df12.columns = ['token{}'.format(i) for i in range(1, num_columns12+1)]
df13 = df11['inputTokens'].map(lambda x:[i['symbol'] for i in x]).apply(pd.Series)
num_columns13 = len(df13.columns)
df13.columns = ['token{}_symbol'.format(i) for i in range(1, num_columns13+1)]
df14 = df11['inputTokens'].map(lambda x:[i['decimals'] for i in x]).apply(pd.Series)
num_columns14 = len(df14.columns)
df14.columns = ['token{}_decimals'.format(i) for i in range(1, num_columns14+1)]

df15 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])
balancerv2_pool1 = pd.concat([df11[['id','totalValueLockedUSD']], df15, df12,df13,df14], axis=1)
balancerv2_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = df21['inputTokens'].map(lambda x:[i['id'] for i in x]).apply(pd.Series)
num_columns22 = len(df22.columns)
df22.columns = ['token{}'.format(i) for i in range(1, num_columns22+1)]
df23 = df21['inputTokens'].map(lambda x:[i['symbol'] for i in x]).apply(pd.Series)
num_columns23 = len(df23.columns)
df23.columns = ['token{}_symbol'.format(i) for i in range(1, num_columns23+1)]
df24 = df21['inputTokens'].map(lambda x:[i['decimals'] for i in x]).apply(pd.Series)
num_columns24 = len(df24.columns)
df24.columns = ['token{}_decimals'.format(i) for i in range(1, num_columns24+1)]

df25 = pd.json_normalize(df21['fees'].apply(pd.Series)[2])
balancerv2_pool2 = pd.concat([df21[['id','totalValueLockedUSD']], df25, df22,df23,df24], axis=1)
balancerv2_pool2

#Combination
balancerv2_pools = pd.concat([balancerv2_pool1,balancerv2_pool2], ignore_index=True)
balancerv2_pools = balancerv2_pools.rename(columns = {'id':'pool_address','totalValueLockedUSD':'tvl','feePercentage':'fee'})
balancerv2_pools
balancerv2_pools = balancerv2_pools[['pool_address','tvl','fee',
                                 'token1','token1_symbol','token1_decimals',
                                 'token2','token2_symbol','token2_decimals',
                                 'token3','token3_symbol','token3_decimals',
                                 'token4','token4_symbol','token4_decimals',
                                 'token5','token5_symbol','token5_decimals',
                                 'token6','token6_symbol','token6_decimals',
                                 'token7','token7_symbol','token7_decimals',
                                 'token8','token8_symbol','token8_decimals']]
Name = pd.Series(["balancerv2" for x in range(len(balancerv2_pools.index))])
balancerv2_pools.insert(loc=0, column='Name', value=Name)
balancerv2_pools

<ipython-input-3-89343c74e88d>:70: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df22 = df21['inputTokens'].map(lambda x:[i['id'] for i in x]).apply(pd.Series)
<ipython-input-3-89343c74e88d>:73: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df23 = df21['inputTokens'].map(lambda x:[i['symbol'] for i in x]).apply(pd.Series)
<ipython-input-3-89343c74e88d>:76: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df24 = df21['inputTokens'].map(lambda x:[i['decimals'] for i in x]).apply(pd.Series)


,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals,...,token5_decimals,token6,token6_symbol,token6_decimals,token7,token7_symbol,token7_decimals,token8,token8_symbol,token8_decimals
0,balancerv2,0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc,573487935.37888266117651532675478,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,ALCX,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,balancerv2,0x5c6ee304399dbdb9c8ef030ab642b10820db8f56,185517497.946307022105492314861531,0.15,0xba100000625a3754423978a60c9317c58a424e3d,BAL,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,balancerv2,0x32296969ef14eb0c6d29669c550d4a0449130230,97495934.811092826232710388038094,0.04,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,balancerv2,0x1e19cf2d73a72ef1332c882f20534b6519be0276,82725839.68579840097432060164336,0.04,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,balancerv2,0x20a61b948e33879ce7f23e535cc7baa3bc66c5a9,39348700.92330163104901975169467226,0.05,0x183015a9ba6ff60230fdeadc3f43b3d788b13e21,R,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,balancerv2,0x0371c272fdd28ac13c434f1ef6b8b52ea3e6d844,0,0.3,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1448,balancerv2,0x0018c32d85d8aebea2efbe0b0f4a4eb9e4f1c8c9,0,0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1449,balancerv2,0xd8cfb615bb6851fbf6cd88b5177bea01936b3f12,-67478.968544918476998942981,2.5,0x12a7530d6f9e1a9b0351d78ab711f7c2c033873a,A Birds Nest,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1450,balancerv2,0xc06764e3def91ca6abdccb18e4c27a87d1f0f6f6,-115261.610789269763871701649,2.22,0x676495371d5107f870e0e7d5afb6fed91f236f21,🐦,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
balancer_token1 = balancerv2_pools[['token1_symbol','token1','token1_decimals']].drop_duplicates()
balancer_token1.columns=[['token_symbol','token_address','token_decimals']]
balancer_token2 = balancerv2_pools[['token2_symbol','token2','token2_decimals']].drop_duplicates()
balancer_token2.columns=[['token_symbol','token_address','token_decimals']]
balancer_token3 = balancerv2_pools[['token3_symbol','token3','token3_decimals']].dropna()
balancer_token3 = balancerv2_pools[['token3_symbol','token3','token3_decimals']].drop_duplicates()
balancer_token3.columns=[['token_symbol','token_address','token_decimals']]
balancer_token4 = balancerv2_pools[['token4_symbol','token4','token4_decimals']].dropna()
balancer_token4 = balancerv2_pools[['token4_symbol','token4','token4_decimals']].drop_duplicates()
balancer_token4.columns=[['token_symbol','token_address','token_decimals']]
balancer_token5 = balancerv2_pools[['token5_symbol','token5','token5_decimals']].dropna()
balancer_token5 = balancerv2_pools[['token5_symbol','token5','token5_decimals']].drop_duplicates()
balancer_token5.columns=[['token_symbol','token_address','token_decimals']]
balancer_token6 = balancerv2_pools[['token6_symbol','token6','token6_decimals']].dropna()
balancer_token6 = balancerv2_pools[['token6_symbol','token6','token6_decimals']].drop_duplicates()
balancer_token6.columns=[['token_symbol','token_address','token_decimals']]
balancer_token7 = balancerv2_pools[['token7_symbol','token7','token7_decimals']].dropna()
balancer_token7 = balancerv2_pools[['token7_symbol','token7','token7_decimals']].drop_duplicates()
balancer_token7.columns=[['token_symbol','token_address','token_decimals']]
balancer_token8 = balancerv2_pools[['token8_symbol','token8','token8_decimals']].dropna()
balancer_token8 = balancerv2_pools[['token8_symbol','token8','token8_decimals']].drop_duplicates()
balancer_token8.columns=[['token_symbol','token_address','token_decimals']]

balancer_token = pd.concat([balancer_token1,balancer_token2,balancer_token3,balancer_token4,balancer_token5,
                           balancer_token6,balancer_token7,balancer_token8], ignore_index=True)
balancer_token = balancer_token.drop_duplicates()
balancer_token.columns = ['symbol','id','decimals']
balancer_token

,symbol,id,decimals
0,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0
1,BAL,0xba100000625a3754423978a60c9317c58a424e3d,18.0
2,wstETH,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,18.0
3,rETH,0xae78736cd615f374d3085123a210448e74fc6393,18.0
4,R,0x183015a9ba6ff60230fdeadc3f43b3d788b13e21,18.0
...,...,...,...
1309,ZCX,0xc52c326331e9ce41f04484d3b5e5648158028804,18.0
1318,SOL,0xd31a59c85ae9d8edefec411d448f90841571b89c,9.0
1321,SAK3,0xe9f84de264e91529af07fa2c746e934397810334,18.0
1324,TRIO,0xf55cd1e399e1cc3d95303048897a680be3313308,8.0


In [32]:
balancer_token8

,token_symbol,token_address,token_decimals
0,NaN,NaN,NaN
143,ENJ,0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c,18.0
194,ROBOT,0xfb5453340c03db5ade474b27e68b6a9c6b2823eb,18.0
310,CRV,0xd533a949740bb3306d119cc777fa900ba034cd52,18.0
331,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0
464,ALCX,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,18.0
537,USDT,0xdac17f958d2ee523a2206206994597c13d831ec7,6.0
587,SOL,0xd31a59c85ae9d8edefec411d448f90841571b89c,9.0
981,BAYC,0xea47b64e1bfccb773a0420247c0aa0a3c1d2e5c5,18.0
1025,AXS,0xf5d669627376ebd411e34b98f19c868c8aba5ada,18.0


In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2-beta',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000
query1 = gql('''
query MyQuery {
  poolContracts(orderBy: pool__totalLiquidity, orderDirection: desc, first: 1000) {
    id
    pool {
      totalLiquidity
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['poolContracts'],max_level=1)
df12 = df11['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns12 = len(df12.columns)
df12.columns = ['token{}'.format(i) for i in range(1, num_columns12+1)]
df13 = df11['pool.tokens'].map(lambda x:[i['symbol'] for i in x]).apply(pd.Series)
num_columns13 = len(df13.columns)
df13.columns = ['token{}_symbol'.format(i) for i in range(1, num_columns13+1)]
df14 = df11['pool.tokens'].map(lambda x:[i['decimals'] for i in x]).apply(pd.Series)
num_columns14 = len(df14.columns)
df14.columns = ['token{}_decimals'.format(i) for i in range(1, num_columns14+1)]


balancer_pool1 = pd.concat([df11[['id','pool.totalLiquidity','pool.swapFee','pool.poolType']], df12,df13,df14], axis=1)

#pool 1001-2000
query2 = gql('''
query {
  poolContracts(first: 1000,
                skip: 1000,
                orderBy: pool__totalLiquidity,
                orderDirection: desc) {
    id
    pool {
      totalLiquidity
      swapFee
      poolType
      tokens {
        address
        symbol
        decimals
      }
    }
  }
}
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['poolContracts'],max_level=1)
df22 = df21['pool.tokens'].map(lambda x:[i['address'] for i in x]).apply(pd.Series)
num_columns22 = len(df22.columns)
df22.columns = ['token{}'.format(i) for i in range(1, num_columns22+1)]
df23 = df21['pool.tokens'].map(lambda x:[i['symbol'] for i in x]).apply(pd.Series)
num_columns23 = len(df23.columns)
df23.columns = ['token{}_symbol'.format(i) for i in range(1, num_columns23+1)]
df24 = df21['pool.tokens'].map(lambda x:[i['decimals'] for i in x]).apply(pd.Series)
num_columns24 = len(df24.columns)
df24.columns = ['token{}_decimals'.format(i) for i in range(1, num_columns24+1)]

balancer_pool2 = pd.concat([df21[['id','pool.totalLiquidity','pool.swapFee','pool.poolType']],df22,df23,df24], axis=1)

#Combination
balancer_pools = pd.concat([balancer_pool1,balancer_pool2], ignore_index=True)
Name = pd.Series(["balancer" for x in range(len(balancer_pools.index))])
balancer_pools.insert(loc=0, column='Name', value=Name)
balancer_pools=balancer_pools.rename(columns = {'id':'pool_address','pool.totalLiquidity':'tvl','pool.swapFee':'fee'})
balancer_pools['fee'] = balancer_pools['fee'].apply(float)*100
x = ['ERC4626Linear','AaveLinear','LiquidityBootstrapping','YearnLinear']
balancer_pools = balancer_pools[~balancer_pools['pool.poolType'].isin(x)]
balancer_pools = balancer_pools.drop('pool.poolType',axis=1)
balancer_pools = balancer_pools[['Name','pool_address','tvl','fee',
                                 'token1','token1_symbol','token1_decimals',
                                 'token2','token2_symbol','token2_decimals',
                                 'token3','token3_symbol','token3_decimals',
                                 'token4','token4_symbol','token4_decimals',
                                 'token5','token5_symbol','token5_decimals',
                                 'token6','token6_symbol','token6_decimals',
                                 'token7','token7_symbol','token7_decimals',
                                 'token8','token8_symbol','token8_decimals']]
balancer_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals,...,token5_decimals,token6,token6_symbol,token6_decimals,token7,token7_symbol,token7_decimals,token8,token8_symbol,token8_decimals
0,balancer,0x5c6ee304399dbdb9c8ef030ab642b10820db8f56,189303344.6805483072042060971283002,1.0000,0xba100000625a3754423978a60c9317c58a424e3d,BAL,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,balancer,0x32296969ef14eb0c6d29669c550d4a0449130230,97013441.3493396307089451233438103,0.0400,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,balancer,0x1e19cf2d73a72ef1332c882f20534b6519be0276,82814624.9638933341053457471186694,0.0400,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,balancer,0x5aee1e99fe86960377de9f88689616916d5dcabe,44288467.50590722639257263198091747,0.0001,0x5aee1e99fe86960377de9f88689616916d5dcabe,wstETH-rETH-sfrxETH-BPT,18.0,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,balancer,0x20a61b948e33879ce7f23e535cc7baa3bc66c5a9,38565468.46457734867490174510480868,0.0500,0x183015a9ba6ff60230fdeadc3f43b3d788b13e21,R,18.0,0x20a61b948e33879ce7f23e535cc7baa3bc66c5a9,R-DAI-BLP,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,balancer,0x062f38735aac32320db5e2dbbeb07968351d7c72,0,0.3000,0x06325440d014e39736583c165c2963ba99faf14e,steCRV,18.0,0xeb1a6c6ea0cd20847150c27b5985fa198b2f90bd,eYyvcrvSTETH-15APR22,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,balancer,0x05a10321f859b6e0c89b01c27c2a8fdaaa7769a8,0,0.3000,0x823e1b82ce1dc147bbdb25a203f046afab1ce918,COIL,18.0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1361,balancer,0x04f5babc66c2464cb9e292f042b6173e2ff627bc,0,0.3000,0xe57425f1598f9b0d6219706b77f4b3da573a3695,BTCBR,18.0,0xfdf888ccd94fc4a3971ff3c5ebaa47cff00445d0,BTCTRADE,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1362,balancer,0x0371c272fdd28ac13c434f1ef6b8b52ea3e6d844,0,0.3000,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Bancor V3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/bancor-v3-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
# df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

bancorv3_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12],axis=1)
bancorv3_pool1

#Combination
bancorv3_pool1.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals']
Name = pd.Series(["bancorv3" for x in range(len(bancorv3_pool1.index))])
bancorv3_pool1.insert(loc=0, column='Name', value=Name)
bancorv3_pool1

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals
0,bancorv3,0xab05cf7c6c3a288cd36326e4f7b8600e7268e344,26028676.37519913389819782568518862,0,0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c,BNT,18
1,bancorv3,0x256ed1d83e3e4efdda977389a5389c3433137dda,23329045.25038203276995491070698781,0.8,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18
2,bancorv3,0x516c164a879892a156920a215855c3416616c46e,12056759.03829377058327170318531889,1.2,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,18
3,bancorv3,0x2ce37087559cbe8022fa5d70a0c502b7ae03f290,6519429.52890183024866622045107187,1.1,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8
4,bancorv3,0xf4d96e11b531b99f1f04fbd77d39859a7689348d,2944640.975187618694277487540112,1,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
...,...,...,...,...,...,...,...
144,bancorv3,0x0ddacad0d1f0ad2d2866f9aa73745d0da203897f,0,0.5,0x960b236a07cf122663c4303350609a66a7b288c0,ANT,18
145,bancorv3,0x0c39c7f6b24ed0e30b453b83cf5d4cf2c5bd3a9e,0,0.5,0xc944e90c64b2c07662a292be6244bdf05cda44a7,GRT,18
146,bancorv3,0x0a95920cd600478e2af01b900e185cab29c92bc2,0,0.5,0x4104b135dbc9609fc1a9490e61369036497660c8,APW,18
147,bancorv3,0x0a8999e064ae4954dae9a7758aa4dc9995218576,0,0.5,0xc12d099be31567add4e4e4d0d45691c3f58f5663,AUC,18


## Curve Finance

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[2])
df15 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[3])
df16 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])


curve_pools = pd.concat([df11[['id','totalValueLockedUSD']],df16,df12,df13,df14,df15],axis=1)

curve_pools.columns = ['pool_address','tvl','fee',
                       'token1','token1_symbol','token1_decimals',
                       'token2','token2_symbol','token2_decimals',
                       'token3','token3_symbol','token3_decimals',
                       'token4','token4_symbol','token4_decimals']
Name = pd.Series(["curve" for x in range(len(curve_pools.index))])
curve_pools.insert(loc=0, column='Name', value=Name)
curve_pools

<ipython-input-6-d777b3614303>:32: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
<ipython-input-6-d777b3614303>:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
<ipython-input-6-d777b3614303>:34: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df14 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[2])
<ipython-input-6-d777b3614303>:35: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df15 = pd.

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals,token3,token3_symbol,token3_decimals,token4,token4_symbol,token4_decimals
0,curve,0xdc24316b9ae028f1497c275eb9192a3ea0f67022,528667012.552435353515795872747,0.01,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,stETH,18.0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18.0,NaN,NaN,NaN,NaN,NaN,NaN
1,curve,0xdcef968d416a41cdac0ed8702fac8128a64241a2,491025080.19842653183012359,0.01,0x853d955acef822db058eb8505911ed77f175b99e,FRAX,18.0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,curve,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,280822133.5342927242747766,0.01,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6.0,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6.0,NaN,NaN,NaN
3,curve,0xa1f8a6807c402e4a15ef4eba36528a3fed24e577,161762401.468295264365850005542846,0.02,0x5e8422345238f34275888049021821e8e08caa1f,frxETH,18.0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18.0,NaN,NaN,NaN,NaN,NaN,NaN
4,curve,0xd632f22692fac7611d2aa1c0d552930d43caed3b,158007280.662171309711942821,0.04,0x6c3f90f043a72fa612cbac8115ee7e52bde6e490,3Crv,18.0,0x853d955acef822db058eb8505911ed77f175b99e,FRAX,18.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,curve,0x02d7f419d731f718bdb22a3b03874c600bec3a40,0,0.44994745,0x3d5a15a9d8ea1d76a32cd70eea882968992d8d95,QUINTA,18.0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6.0,NaN,NaN,NaN,NaN,NaN,NaN
706,curve,0x02ac4107893ba767177d69851dcd87bedc63ab22,0,0.04,0xa0d69e286b938e21cbf7e51d71f6a4c8918f482f,eUSD,18.0,0xacdf0dba4b9839b96221a8487e9ca660a48212be,hyUSD,18.0,NaN,NaN,NaN,NaN,NaN,NaN
707,curve,0x02914596cad247c86e8f7d8464d1b3dbd0cec86e,0,0.04,0x6c3f90f043a72fa612cbac8115ee7e52bde6e490,3Crv,18.0,0x97de57ec338ab5d51557da3434828c5dbfada371,eUSD,18.0,NaN,NaN,NaN,NaN,NaN,NaN
708,curve,0x01fe650ef2f8e2982295489ae6adc1413bf6011f,0,0.04,0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415,tTCR,18.0,0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050,TCR,18.0,NaN,NaN,NaN,NaN,NaN,NaN


## Pancakeswap V3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/steegecs/pancakeswap-v3-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pools = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
pancakeswap_pools

pancakeswap_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap" for x in range(len(pancakeswap_pools.index))])
pancakeswap_pools.insert(loc=0, column='Name', value=Name)
pancakeswap_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap,0x6ca298d2983ab03aa1da7679389d955a4efee15c,13057995.60672205404361466562516,0.05,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
1,pancakeswap,0x04c8577958ccc170eb3d2cca76f9d51bc6e42d8f,10659662.141051,0.01,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
2,pancakeswap,0x2201d2400d30bfd8172104b4ad046d019ca4e7bd,7060653.45597599494010409609745888,0.05,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
3,pancakeswap,0x6c2348deec1508724c0207a84d21cc5ee88231c8,6414939.623198806413012684,0.01,0x0000000000085d4780b73119b644ae5ecd22b376,TUSD,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
4,pancakeswap,0x9b5699d18dff51fc65fb8ad6f70d93287c36349f,3247279.719253558445068499915182477,0.25,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
90,pancakeswap,0x9c29fbf5a63aca6d727cb286952cb5328971a2c5,0,0.25,0x2dd4239206e6a4933e0cfe139950042ba6dcc7f0,USDT,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
91,pancakeswap,0x7778797342652bd27b365962ffc7f6ece356eb57,0,0.05,0xd84787a01b0cad89fbca231e6960cc0f3f18df34,MOCHI,18,0xdb19f2052d2b1ad46ed98c66336a5daadeb13005,RUBY,18
92,pancakeswap,0x75ba833e5546cfb897c415f5cc453ece3951419e,0,0.25,0x1731ad9f2db8051e8b76aeda9ca40685127fa4bd,LV,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
93,pancakeswap,0x6c62045e0a1cb24c6a67ff8723d61f49c2399b22,0,1,0x392b7dd300715e2810d58e611057a32775096715,USDT,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


## Saddle Finance

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/saddle-finance-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[2])
df15 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[3])
df16 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])


saddle_pools = pd.concat([df11[['id','totalValueLockedUSD']],df16,df12,df13,df14,df15],axis=1)

saddle_pools.columns = ['pool_address','tvl','fee',
                       'token1','token1_symbol','token1_decimals',
                       'token2','token2_symbol','token2_decimals',
                       'token3','token3_symbol','token3_decimals',
                       'token4','token4_symbol','token4_decimals']
Name = pd.Series(["saddle" for x in range(len(saddle_pools.index))])
saddle_pools.insert(loc=0, column='Name', value=Name)
saddle_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals,token3,token3_symbol,token3_decimals,token4,token4_symbol,token4_decimals
0,saddle,0xa6018520eaacc06c30ff2e1b3ee2c7c22e64196a,989169.185945616676218305699328,0.04,0x0100546f2cd4c9d97f798ffc9755e47865ff7ee6,alETH,18,0x5e74c9036fb86bd7ecdcb084a0673efc32ea31cb,sETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,NaN,NaN,NaN
1,saddle,0x13cc34aa8037f722405285ad2c82fe570bfa2bdc,659558.9655922977139095,0.04,0x853d955acef822db058eb8505911ed77f175b99e,FRAX,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,NaN,NaN,NaN,NaN,NaN,NaN
2,saddle,0xacb83e0633d6605c5001e2ab59ef3c745547c8c7,534461.440296493078649038,0.03,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6.0,NaN,NaN,NaN
3,saddle,0x4f6a43ad7cba042606decaca730d4ce0a57ac62e,218623.87408982419444718227495,0.04,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0x8daebade922df735c38c80c7ebd708af50815faa,TBTC,18,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,renBTC,8.0,0xfe18be6b3bd88a2d2a7f928d00292e7a9963cfc6,sBTC,18.0
4,saddle,0xb62222b941e9b652be3632eea062cb0ff66b1d1c,199455.893074384222047641,0.04,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xad3e3fc59dff318beceaab7d00eb4f68b1ecf195,wCUSD,18.0,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6.0
5,saddle,0xdf3309771d2bf82cb2b6c56f9f5365c8bd97c4f2,113405.72856667509803075287,0.04,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,renBTC,8,0xfe18be6b3bd88a2d2a7f928d00292e7a9963cfc6,sBTC,18.0,NaN,NaN,NaN
6,saddle,0xc69ddcd4dfef25d8a793241834d4cc4b3668ead6,95337.260198725310403288,0.04,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,LUSD,18,0x853d955acef822db058eb8505911ed77f175b99e,FRAX,18,0x956f47f50a910163d8bf957cf5846d573e7f87ca,FEI,18.0,0xbc6da0fe9ad5f3b0d58160288917aa56653660e9,alUSD,18.0
7,saddle,0xfb516cf3710fc6901f2266aaeb8834cf5e4e9558,39968.061526905654855096,0.04,0x853d955acef822db058eb8505911ed77f175b99e,FRAX,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xbc6da0fe9ad5f3b0d58160288917aa56653660e9,alUSD,18.0,NaN,NaN,NaN
8,saddle,0xfa9ed0309bf79eb84c847819f0b3cb84f6d351af,38382.53280126425572589226,0.04,0x18084fba666a33d37592fa2633fd49a74dd93a88,tBTC,18,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,renBTC,8.0,0xfe18be6b3bd88a2d2a7f928d00292e7a9963cfc6,sBTC,18.0
9,saddle,0xf74ebe6e5586275dc4ced78f5dbef31b1efbe7a5,21122.183847057715313248087775,0.04,0x18084fba666a33d37592fa2633fd49a74dd93a88,tBTC,18,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,renBTC,8.0,0xfe18be6b3bd88a2d2a7f928d00292e7a9963cfc6,sBTC,18.0


## Sushiswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

sushiswap_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
sushiswap_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df24 = pd.json_normalize(df21['fees'].apply(pd.Series)[2])

sushiswap_pool2 = pd.concat([df21[['id','totalValueLockedUSD']],df24,df22,df23],axis=1)
sushiswap_pool2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 2000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df34 = pd.json_normalize(df31['fees'].apply(pd.Series)[2])

sushiswap_pool3 = pd.concat([df31[['id','totalValueLockedUSD']],df34,df32,df33],axis=1)
sushiswap_pool3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 3000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df44 = pd.json_normalize(df41['fees'].apply(pd.Series)[2])

sushiswap_pool4 = pd.concat([df41[['id','totalValueLockedUSD']],df44,df42,df43],axis=1)
sushiswap_pool4

#Combination
sushiswap_pools = pd.concat([sushiswap_pool1,sushiswap_pool2,sushiswap_pool3,sushiswap_pool4], ignore_index=True)
sushiswap_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["sushiswap" for x in range(len(sushiswap_pools.index))])
sushiswap_pools.insert(loc=0, column='Name', value=Name)
sushiswap_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,sushiswap,0x6a091a3406e0073c3cd6340122143009adac0eda,49892967.68456616274500209181801846,0.3,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,ILV,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,sushiswap,0xceff51756c56ceffca006cd410b03ffc46dd3a58,21129030.07008055746213335257186228,0.3,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,sushiswap,0x06da0fd433c1a5d7a4faa01111c044910a184553,18603769.383784,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
3,sushiswap,0x397ff1542f962076d0bfe58ea045ffa2d347aca0,17188445.17959009317830642974208844,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,sushiswap,0x3e04863dba602713bb5d0edbf7db7c3a9a2b6027,11414282.42931420939595662354816804,0.3,0x55c08ca52497e2f1534b59e2917bf524d4765257,UwU,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
3758,sushiswap,0x04c79ea4498f400b713e4fa2966694e9172eab9e,0,0.3,0x93747501f46ae40b8a4b8f1a1529696ae24ea04e,KETH,18,0xcb5f72d37685c3d5ad0bb5f982443bc8fcdf570e,ROOT,18
3759,sushiswap,0x04771d6e1c4b1a6912c74ec53e6d1e83b45e4e9b,0,0.3,0x9470a37d562d6a4052f1c633848c2648460d688e,EARN,18,0xb220d53f7d0f52897bcf25e47c4c3dc0bac344f8,EUSD,18
3760,sushiswap,0x033f4a33823595a6dd9df0672fd94de32c65c415,0,0.3,0x31b595e7cfdb624d10a3e7a562ed98c3567e3865,stZEN,8,0xdfe66b14d37c77f4e9b180ceb433d1b164f0281d,stETH,18
3761,sushiswap,0x0313ba7d9b5e1df207aca261cf381405a0e31a52,0,0.3,0x850fe284f917eb6dfcfe452348bead903aa63073,VGX/NRC,18,0xc9cb53b48a2f3a9e75982685644c1870f1405ccb,MIC/Tether USD,18


## Sushiswap V3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/sushiswap-v3-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

sushiswapv3_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
sushiswapv3_pool1

#Combination
sushiswapv3_pools = pd.concat([sushiswapv3_pool1], ignore_index=True)
sushiswapv3_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["sushiswapv3" for x in range(len(sushiswapv3_pools.index))])
sushiswapv3_pools.insert(loc=0, column='Name', value=Name)
sushiswapv3_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,sushiswapv3,0x35644fb61afbc458bf92b15add6abc1996be5014,463952.5115610029492293675410583356,0.05,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,sushiswapv3,0x801ccfae9d2c77893b545e8d0e4637c055cd26cb,289533.382530079730010475286333976,0.05,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,sushiswapv3,0x87c7056bbe6084f03304196be51c6b90b6d85aa2,265436.9486862955228928284332342762,0.3,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,SUSHI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
3,sushiswapv3,0xfa6e8e97ececdc36302eca534f63439b1e79487b,263790.315334,0.01,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
4,sushiswapv3,0x72c2178e082fedb13246877b5aa42ebce1b72218,218884.0136806142932713110206559608,0.05,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
5,sushiswapv3,0x31ac258b911af9a0d2669ebdfc4e39d92e96b772,135448.48017602090236398,0.01,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
6,sushiswapv3,0x84b2f2d7e1577d6eebbb2239157cd2d644959ba3,109370.8489043674159297936004543469,0.01,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
7,sushiswapv3,0x7c0a81c0cd5c976f88fa102063ec12c388da9408,50738.98061825504871505203662821923,0.3,0x5325d78529ef74105004bd4114cc569303ce8b5f,SCHIZO,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
8,sushiswapv3,0x22f25e528d734ea80b683abd15716c236bf397f2,43532.0978995778784191358887,1,0x50d1c9771902476076ecfc8b2a83ad6b9355a4c9,FTX Token,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
9,sushiswapv3,0x5f11853100774087030845d4c47a9d42b429ed83,12825.52592253948889932467763736465,0.3,0x5b7533812759b45c2b44c19e320ba2cd2681b542,AGIX,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


## Uniswap V2

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v2-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

uniswapv2_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
uniswapv2_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df24 = pd.json_normalize(df21['fees'].apply(pd.Series)[2])

uniswapv2_pool2 = pd.concat([df21[['id','totalValueLockedUSD']],df24,df22,df23],axis=1)
uniswapv2_pool2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 2000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])
df34 = pd.json_normalize(df31['fees'].apply(pd.Series)[2])

uniswapv2_pool3 = pd.concat([df31[['id','totalValueLockedUSD']],df34,df32,df33],axis=1)
uniswapv2_pool3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 3000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df44 = pd.json_normalize(df41['fees'].apply(pd.Series)[2])

uniswapv2_pool4 = pd.concat([df41[['id','totalValueLockedUSD']],df44,df42,df43],axis=1)
uniswapv2_pool4

#pool 4001-5000
query5 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 4000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response5 = client.execute(query5)

df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])
df54 = pd.json_normalize(df51['fees'].apply(pd.Series)[2])

uniswapv2_pool5 = pd.concat([df51[['id','totalValueLockedUSD']],df54,df52,df53],axis=1)
uniswapv2_pool5

#pool 5001-6000
query6 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 5000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response6 = client.execute(query6)

df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])
df64 = pd.json_normalize(df61['fees'].apply(pd.Series)[2])

uniswapv2_pool6 = pd.concat([df61[['id','totalValueLockedUSD']],df64,df62,df63],axis=1)
uniswapv2_pool6

#Combination
uniswapv2_pools = pd.concat([uniswapv2_pool1,uniswapv2_pool2,uniswapv2_pool2,uniswapv2_pool4,uniswapv2_pool5,uniswapv2_pool6], ignore_index=True)
uniswapv2_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["uniswapv2" for x in range(len(uniswapv2_pools.index))])
uniswapv2_pools.insert(loc=0, column='Name', value=Name)
uniswapv2_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,uniswapv2,0xd089fe8833d467aa9884f2e7ee9502c830fb5f88,16312424158.32506700009042706672057,0.3,0x528fcd6a53a2b450cdf0b8b0f974f43cf612ed6d,FireBlood,9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,uniswapv2,0x21b8065d10f73ee2e260e5b47d3344d3ced7596e,121872057.7914891647243528360225841,0.3,0x66a0f676479cee1d7373f3dc2e2952778bff5bd6,WISE,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,uniswapv2,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,60508026.08827,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
3,uniswapv2,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,59284795.80443970754147939141591622,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,uniswapv2,0x61b62c5d56ccd158a38367ef2f539668a06356ab,29736151.48592,0.3,0xb5fe099475d3030dde498c3bb6f3854f762a48ad,FNK,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
...,...,...,...,...,...,...,...,...,...,...
5995,uniswapv2,0x85f05e91476ff0400e45ce7dc25eda5abe2277ec,9853.330033261599295048950551507413,0.3,0x8025d6c18807c4ff46f316c1942462b907119c7e,PEPEGA,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5996,uniswapv2,0x13564deb36f07aaa1ec98f45b8ffe80f6357f6d3,9851.095784445737439221310333840478,0.3,0x4a2c98723cc127ca72d27ca61636c238313889f9,gae,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5997,uniswapv2,0x415a442376cd33ce99e9bdad67e3aca560eca798,9849.380748575003141815994040612811,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xfe1e07badbaa075824c28569454212c264fe9c6a,LO,18
5998,uniswapv2,0x970de3b3fffb141610274e13d5e4d4d3c39e1491,9846.383548075755678482763624659599,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdeb69f29e46f81de5ed2c13eeafe9d3c29b6b878,YOSHI,18


## Uniswap V3

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

uniswapv3_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
uniswapv3_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df24 = pd.json_normalize(df21['fees'].apply(pd.Series)[2])

uniswapv3_pool2 = pd.concat([df21[['id','totalValueLockedUSD']],df24,df22,df23],axis=1)
uniswapv3_pool2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 2000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df31['inputTokens'].apply(pd.Series)[1])
df34 = pd.json_normalize(df31['fees'].apply(pd.Series)[2])

uniswapv3_pool3 = pd.concat([df31[['id','totalValueLockedUSD']],df34,df32,df33],axis=1)
uniswapv3_pool3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 3000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df44 = pd.json_normalize(df41['fees'].apply(pd.Series)[2])

uniswapv3_pool4 = pd.concat([df41[['id','totalValueLockedUSD']],df44,df42,df43],axis=1)
uniswapv3_pool4

#pool 4001-5000
query5 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 4000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response5 = client.execute(query5)

df51 = pd.json_normalize(response5['liquidityPools'],max_level=1)
df52 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[0])
df53 = pd.json_normalize(df51['inputTokens'].apply(pd.Series)[1])
df54 = pd.json_normalize(df51['fees'].apply(pd.Series)[2])

uniswapv3_pool5 = pd.concat([df51[['id','totalValueLockedUSD']],df54,df52,df53],axis=1)
uniswapv3_pool5

#pool 5001-6000
query6 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 5000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response6 = client.execute(query6)

df61 = pd.json_normalize(response6['liquidityPools'],max_level=1)
df62 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[0])
df63 = pd.json_normalize(df61['inputTokens'].apply(pd.Series)[1])
df64 = pd.json_normalize(df61['fees'].apply(pd.Series)[2])

uniswapv3_pool6 = pd.concat([df61[['id','totalValueLockedUSD']],df64,df62,df63],axis=1)
uniswapv3_pool6

#Combination
uniswapv3_pools = pd.concat([uniswapv3_pool1,uniswapv3_pool2,uniswapv3_pool3,uniswapv3_pool4,uniswapv3_pool5,uniswapv3_pool6], ignore_index=True)
uniswapv3_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["uniswapv3" for x in range(len(uniswapv3_pools.index))])
uniswapv3_pools.insert(loc=0, column='Name', value=Name)
uniswapv3_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,uniswapv3,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,240812953.2068146778410408323855799,0.05,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,uniswapv3,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,223854617.9946815606037092081402855,0.3,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,uniswapv3,0x3416cf6c708da44db2624d63ea0aaef7113527c6,122551727.997407,0.01,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
3,uniswapv3,0x5777d92f208679db4b9778590fa3cab3ac9e2168,107081591.444006696522901334,0.01,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
4,uniswapv3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,95213804.13565069247631876116742788,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
5995,uniswapv3,0x9c2dc06da074919537ebf0bd1e85fc815dea797c,0.1932552513354979323393175470750794,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xebe4a49df7885d015329c919bf43e6460a858f1e,SHK,18
5996,uniswapv3,0x87e4891a187c50ec7d02fc4ae69db5faa130f9a6,0.1922450930385228216925705354674659,0.3,0x5edc0908952dcacc1b65bc924d3f15226f108b43,SCODA,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5997,uniswapv3,0x50185120159756c700ab6eac6b6480a053af6180,0.1917294019553060215396179438149749,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xe00a182284098e9c2ba89634544d51b0179c4c92,CASH,18
5998,uniswapv3,0x846c0a108ff642ad505e394fdd2f2562c2137fb8,0.1911426374093112455130542562881588,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xd832d3b46e70c7b8b9de15715c9c119b0fa99280,NDT,18


## Maverick Protocol

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/maverick-protocol/maverick-mainnet-data-2',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000
query1 = gql('''
query {
  pools(first: 1000, orderBy: balanceUSD, orderDirection: desc) {
    id
    balanceUSD
    fee
    tokenA {
      id
      symbol
      decimals
    }
    tokenB {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

maverick_pool1 = pd.json_normalize(response1['pools'],max_level=1)
maverick_pool1

#Combination
maverick_pools = pd.concat([maverick_pool1], ignore_index=True)
maverick_pools.columns = ['pool_address','tvl','fee','token1','token1_symbol','token1_decimals','token2','token2_symbol','token2_decimals']
Name = pd.Series(["maverick" for x in range(len(maverick_pools.index))])
maverick_pools.insert(loc=0, column='Name', value=Name)
maverick_pools['fee'] = maverick_pools['fee'].apply(lambda x: float(x)*100)
maverick_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,maverick,0x0ce176e1b11a8f88a4ba2535de80e81f88592bad,20250371.92427534039199317918926007,0.020,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xf951e335afb289353dc249e82926178eac7ded78,swETH,18
1,maverick,0x0eb1c92f9f5ec9d817968afddb4b46c564cdedbe,9792118.736760201865329061614110587,0.010,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,maverick,0xd0b2f5018b5d22759724af6d4281ac0b13266360,3271035.934180141144253212,0.002,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
3,maverick,0x7064a1ea78961ef05a510e86540235a4a2e64ca4,2000652.634497582344278322704858297,0.040,0x0ae38f7e10a43b5b2fb064b42a2f4514cba909ef,unshETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,maverick,0x6c6fc818b25df89a8ada8da5a43669023bad1f4c,1960804.074212027752864317287448392,0.030,0x5f98805a4e8be255a32880fdec7f6728c6568ba0,LUSD,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
...,...,...,...,...,...,...,...,...,...,...
113,maverick,0x1970523c241756e24d546dc4bfffc1f436ab9671,0,0.250,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
114,maverick,0x12378796271f2c5fb1a3206525c4b49a204a8487,0,0.010,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
115,maverick,0x123683d168d8615b73bd7c85a1dc947414965d53,0,0.100,0x183015a9ba6ff60230fdeadc3f43b3d788b13e21,R,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
116,maverick,0x05b20bece2969edef0359d41d617ccfbf47f2247,0,0.030,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


## Addition for Sushiswap

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/dmelotik/sushiswap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

sushiswap_pool1 = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
sushiswap_pool1

#pool 1001-2000
query2 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response2 = client.execute(query2)

df21 = pd.json_normalize(response2['liquidityPools'],max_level=1)
df22 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df23 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df24 = pd.json_normalize(df21['fees'].apply(pd.Series)[2])

sushiswap_pool2 = pd.concat([df21[['id','totalValueLockedUSD']],df24,df22,df23],axis=1)
sushiswap_pool2

#pool 2001-3000
query3 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 2000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response3 = client.execute(query3)

df31 = pd.json_normalize(response3['liquidityPools'],max_level=1)
df32 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[0])
df33 = pd.json_normalize(df21['inputTokens'].apply(pd.Series)[1])
df34 = pd.json_normalize(df31['fees'].apply(pd.Series)[2])

sushiswap_pool3 = pd.concat([df31[['id','totalValueLockedUSD']],df34,df32,df33],axis=1)
sushiswap_pool3

#pool 3001-4000
query4 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000,
                                skip: 3000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response4 = client.execute(query4)

df41 = pd.json_normalize(response4['liquidityPools'],max_level=1)
df42 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[0])
df43 = pd.json_normalize(df41['inputTokens'].apply(pd.Series)[1])
df44 = pd.json_normalize(df41['fees'].apply(pd.Series)[2])

sushiswap_pool4 = pd.concat([df41[['id','totalValueLockedUSD']],df44,df42,df43],axis=1)
sushiswap_pool4

#Combination
sushiswap_pools = pd.concat([sushiswap_pool1,sushiswap_pool2,sushiswap_pool3,sushiswap_pool4], ignore_index=True)
sushiswap_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["sushiswap" for x in range(len(sushiswap_pools.index))])
sushiswap_pools.insert(loc=0, column='Name', value=Name)
sushiswap_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,sushiswap,0x89fa2fd7dd529d70912b434269843389c5575822,153651635.6908276523666029337553719,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xc7924bf912ebc9b92e3627aed01f816629c7e400,QAZ,18
1,sushiswap,0x9ea56195cfeffde9957df9f33ab4642e202fb40e,139769985.7266217217151418509747452,0.3,0x948a9bb69d1d1202c160d26804aefff0634a492e,scUSD,8,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
2,sushiswap,0x4234fdc6c354805236498bce11420a7ceae7ffcf,78784189.7183392226887442822447379,0.3,0x948a9bb69d1d1202c160d26804aefff0634a492e,scUSD,8,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
3,sushiswap,0xd21dfb6e05e60e14cef9a5cf875065bccd5ed790,64414029.89895609341902787792654453,0.3,0x53a1e9912323b8016424d6287286e3b6de263f76,PTT,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,sushiswap,0x6a091a3406e0073c3cd6340122143009adac0eda,49893776.61226510237918983044261189,0.3,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,ILV,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
3758,sushiswap,0x033f4a33823595a6dd9df0672fd94de32c65c415,0,0.3,0x31b595e7cfdb624d10a3e7a562ed98c3567e3865,stZEN,8,0xdfe66b14d37c77f4e9b180ceb433d1b164f0281d,stETH,18
3759,sushiswap,0x031ea09922fb6dc929985e5cb280c2abbbd684fc,0,0.3,0x41c028a4c1f461ebfc3af91619b240004ebad216,TACO,18,0xcaaa93712bdac37f736c323c93d4d5fdefcc31cc,CRD,18
3760,sushiswap,0x0313ba7d9b5e1df207aca261cf381405a0e31a52,0,0.3,0x850fe284f917eb6dfcfe452348bead903aa63073,VGX/NRC,18,0xc9cb53b48a2f3a9e75982685644c1870f1405ccb,MIC/Tether USD,18
3761,sushiswap,0x02cb152d029f06a2ae796eae564e0ae740a8b7ca,0,0.3,0x3e9bc21c9b189c09df3ef1b824798658d5011937,LINA,18,0x4b4d2e899658fb59b1d518b68fe836b100ee8958,MIS,18


## ShibaSwap
https://shibaswap-com.github.io/
Didn't find gas fee

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/shibaswap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

shibaswap_pool1 = pd.json_normalize(response1['pairs'],max_level=1)
shibaswap_pool1

#Combination
shibaswap_pools = pd.concat([shibaswap_pool1], ignore_index=True)
shibaswap_pools.columns = ['pool_address',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["shibaswap" for x in range(len(shibaswap_pools.index))])
shibaswap_pools.insert(loc=0, column='Name', value=Name)
shibaswap_pools.insert(loc=2, column='fee', value=0.3)
shibaswap_pools

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,shibaswap,0x6488eeab0fb00f96f41596d08f763ef91a993495,0.3,0x30a242225464ee334b6500e7d53feccfafe2a1a9,PAWBOT,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,shibaswap,0xf8ce803c2cac24ed8dcef34ba772fc0ecdfdbb4a,0.3,0x50d823dd2a5a7cbb21db6392c3a94845a1409ef0,TAIL,9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,shibaswap,0x7cc067c3ed66daa3f3816aaeb5feb840690dd4a3,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xf452ca9891eff349af6b59c0708dfdcbde783a10,PAWBOT,18
3,shibaswap,0x73fd8a9390d14463e2de2c668d72f82359f1dfa8,0.3,0x214506973dc5d8032465f1f46320568a429318e0,PAWBOT,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,shibaswap,0x288babe50ae108f8cf72e72dafb7261d7e20e526,0.3,0x6d47ee70bd21739832e7b0a9f5a683c7213a03c8,LEASHING,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...
913,shibaswap,0x6cbefa95e42960e579c2a3058c05c6a08e2498e9,0.3,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,SUSHI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
914,shibaswap,0x20e95253e54490d8d30ea41574b24f741ee70201,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
915,shibaswap,0x86cccda3799a43fbceb4cec6ad8b028f5bf8b3dd,0.3,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
916,shibaswap,0x57654ae132413e81459ad2ae70c2570a9b89fb53,0.3,0x27c70cd1946795b66be9d954418546998b546634,LEASH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/shibaswap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pairs(first: 1000, orderBy: syncAtTimestamp, orderDirection: desc) {
    id
    syncAtTimestamp
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

shibaswap_pool1 = pd.json_normalize(response1['pairs'],max_level=1)
shibaswap_pool1

#Combination
shibaswap_pools = pd.concat([shibaswap_pool1], ignore_index=True)
shibaswap_pools.columns = ['pool_address','syncAtTimestamp',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["shibaswap" for x in range(len(shibaswap_pools.index))])
shibaswap_pools.insert(loc=0, column='Name', value=Name)
# shibaswap_pools.insert(loc=3, column='fee', value=0.2)
shibaswap_pools

,Name,pool_address,syncAtTimestamp,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,shibaswap,0xefb47fcfcad4f96c83d4ca676842fb03ef20a477,1689305171,0x9813037ee2218799597d83d4a5b6f3b6778218d9,BONE,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,shibaswap,0x9d406c4067a53f65de1a8a9273d55bfea5870a75,1689304907,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,shibaswap,0xff64cb7ba5717a10dabc4be3a41acd2c2f95ee22,1689304547,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIB,18,0x9813037ee2218799597d83d4a5b6f3b6778218d9,BONE,18
3,shibaswap,0xcf6daab95c476106eca715d48de4b13287ffdeaa,1689304463,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIB,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
4,shibaswap,0x57654ae132413e81459ad2ae70c2570a9b89fb53,1689304415,0x27c70cd1946795b66be9d954418546998b546634,LEASH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...
903,shibaswap,0x411de09cc52d677a7cb0565ff1d1a0bbd40cab18,0,0x40db9c09ef84fbb51bc64d391336dc1ba199e829,PRETEND,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
904,shibaswap,0x4081d5c7c25bc1b076b5d363f23357fda849a580,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xe8274859eb93f2e508ebf0b8cf1d96f8495c3b3c,CORGI,18
905,shibaswap,0x33d5e1562dff59ca240c4281456673fa976c021a,0,0x07ab179ada2535a8ef2a6f15571b1dc7b6c3850b,LAST,9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
906,shibaswap,0x300408beb437748a81aaeb4def4003a0781177ea,0,0xb0004c46a7efd45744e02e5b8d487819264e1ba6,BNEYMA,9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/messari/pancakeswap-v3-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
        liquidityPools(orderBy: totalValueLockedUSD,
                                orderDirection: desc,
                                first: 1000){
            id
            totalValueLockedUSD
            fees {
              feePercentage
            }
            inputTokens {
              id
              symbol
              decimals
    }
  }
      }
''')

response1 = client.execute(query1)

df11 = pd.json_normalize(response1['liquidityPools'],max_level=1)
df12 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[0])
df13 = pd.json_normalize(df11['inputTokens'].apply(pd.Series)[1])
df14 = pd.json_normalize(df11['fees'].apply(pd.Series)[2])

pancakeswap_pools = pd.concat([df11[['id','totalValueLockedUSD']],df14,df12,df13],axis=1)
pancakeswap_pools

pancakeswap_pools.columns = ['pool_address','tvl','fee',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["pancakeswap" for x in range(len(pancakeswap_pools.index))])
pancakeswap_pools.insert(loc=0, column='Name', value=Name)
pancakeswap_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,pancakeswap,0x6ca298d2983ab03aa1da7679389d955a4efee15c,13072599.5899237476571881499738,0.05,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
1,pancakeswap,0x04c8577958ccc170eb3d2cca76f9d51bc6e42d8f,10659662.218571,0.01,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
2,pancakeswap,0x2201d2400d30bfd8172104b4ad046d019ca4e7bd,7060653.45597599494010409609745888,0.05,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
3,pancakeswap,0x6c2348deec1508724c0207a84d21cc5ee88231c8,6414939.623198806413012684,0.01,0x0000000000085d4780b73119b644ae5ecd22b376,TUSD,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
4,pancakeswap,0x9b5699d18dff51fc65fb8ad6f70d93287c36349f,3237985.028185550801915016395947416,0.25,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
90,pancakeswap,0x9c29fbf5a63aca6d727cb286952cb5328971a2c5,0,0.25,0x2dd4239206e6a4933e0cfe139950042ba6dcc7f0,USDT,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
91,pancakeswap,0x7778797342652bd27b365962ffc7f6ece356eb57,0,0.05,0xd84787a01b0cad89fbca231e6960cc0f3f18df34,MOCHI,18,0xdb19f2052d2b1ad46ed98c66336a5daadeb13005,RUBY,18
92,pancakeswap,0x75ba833e5546cfb897c415f5cc453ece3951419e,0,0.25,0x1731ad9f2db8051e8b76aeda9ca40685127fa4bd,LV,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
93,pancakeswap,0x6c62045e0a1cb24c6a67ff8723d61f49c2399b22,0,1,0x392b7dd300715e2810d58e611057a32775096715,USDT,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


### CRODefi


In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/unchase/crodefiswap-ethereum',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pairs(first: 1000, orderBy: createdAtTimestamp, orderDirection: desc) {
    id
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

crodefi_pool1 = pd.json_normalize(response1['pairs'],max_level=1)
crodefi_pool1

#Combination
crodefi_pools = pd.concat([crodefi_pool1], ignore_index=True)
crodefi_pools.columns = ['pool_address',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["crodefi" for x in range(len(crodefi_pools.index))])
crodefi_pools.insert(loc=0, column='Name', value=Name)
crodefi_pools.insert(loc=2, column='fee', value=0.3)
crodefi_pools

,Name,pool_address,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,crodefi,0xfdbfb4239935a15c2c348400570e34de3b044c5f,0.3,0x608a911ff9f2cf21e92f48f0c2f7a70390aa8f7e,Ogopogo,9,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
1,crodefi,0xe6ce0226859f99c095c5b405bf187dc3c55ab4d8,0.3,0x0ed7787b6f5ee00976fa376d04d5b835e48fc42b,BTC,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
2,crodefi,0x46fff8a2f03eeb26eb8c2cf175d006eda0195e14,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xe57425f1598f9b0d6219706b77f4b3da573a3695,BTCBR,18
3,crodefi,0x27c64b56e3c0be134b348a0ad047c0ec92858324,0.3,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xfdf888ccd94fc4a3971ff3c5ebaa47cff00445d0,BTCTRADE,18
4,crodefi,0x3af08d9e697a99afd23ba51adbacee92fe7ec5ef,0.3,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,8,0xe57425f1598f9b0d6219706b77f4b3da573a3695,BTCBR,18
...,...,...,...,...,...,...,...,...,...
94,crodefi,0x66c4a3f62f7be4aa125549f18eef3ec2cb9065a5,0.3,0x514910771af9ca656af840dff83e8264ecf986ca,LINK,18,0xa0b73e1ff0b80914ab6fe0444e65848c4c34450b,CRO,8
95,crodefi,0x74c99f3f5331676f6aec2756e1f39b4fc029a83e,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
96,crodefi,0x9ef7e917fb41cc02f78a5c99b42f497ed8979350,0.3,0xa0b73e1ff0b80914ab6fe0444e65848c4c34450b,CRO,8,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
97,crodefi,0x3aa370aacf4cb08c7e1e7aa8e8ff9418d73c7e0f,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18


## All Pools

In [ ]:
eth_pools = pd.concat([balancerv2_pools,bancorv3_pool1,curve_pools,pancakeswap_pools,saddle_pools,sushiswap_pools,sushiswapv3_pools,uniswapv2_pools,uniswapv3_pools,maverick_pools,shibaswap_pools,crodefi_pools])
eth_pools.head(20)

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals,...,token5_decimals,token6,token6_symbol,token6_decimals,token7,token7_symbol,token7_decimals,token8,token8_symbol,token8_decimals
0,balancerv2,0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc,573487935.37888266117651532675478,1,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,ALCX,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,balancerv2,0x5c6ee304399dbdb9c8ef030ab642b10820db8f56,185517497.946307022105492314861531,0.15,0xba100000625a3754423978a60c9317c58a424e3d,BAL,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,balancerv2,0x32296969ef14eb0c6d29669c550d4a0449130230,97495934.811092826232710388038094,0.04,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,balancerv2,0x1e19cf2d73a72ef1332c882f20534b6519be0276,82725839.68579840097432060164336,0.04,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,balancerv2,0x20a61b948e33879ce7f23e535cc7baa3bc66c5a9,39348700.92330163104901975169467226,0.05,0x183015a9ba6ff60230fdeadc3f43b3d788b13e21,R,18.0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,balancerv2,0x5aee1e99fe86960377de9f88689616916d5dcabe,26331426.087991545249320376555681,0.04,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,wstETH,18.0,0xac3e018457b222d93114458476f3e3416abbe38f,sfrxETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,balancerv2,0x1ee442b5326009bb18f2f472d3e0061513d1a0ff,19061172.018975339772435492523073,0.25,0x3472a5a71965499acd81997a54bba8d852c6e53d,BADGER,18.0,0xae78736cd615f374d3085123a210448e74fc6393,rETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,balancerv2,0x02d928e68d8f10c0358566152677db51e1e2dc8c,17437575.971974551029578803697804,0.04,0x60d604890feaa0b5460b28a424407c24fe89374a,bb-a-WETH,18.0,0xf951e335afb289353dc249e82926178eac7ded78,swETH,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,balancerv2,0xa27e92f81b42feb6e0d507d1c0f75a5284e921e6,16033778.22619785525118291156377865,0.38,0x48f07301e9e29c3c38a80ae8d9ae771f224f1054,XZAR,18.0,0x4fabb145d64652a948d72533023f6e7a623c7c53,BUSD,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,balancerv2,0xfebb0bbf162e64fb9d0dfe186e517d84c395f016,15508374.26507313348885984874461,0.001,0x6667c6fa9f2b3fc1cc8d85320b62703d938e4385,bb-a-DAI,18.0,0xa1697f9af0875b63ddc472d6eebada8c1fab8568,bb-a-USDT,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
eth_pools.to_csv('eth_subgraph.csv')

## DODO
volumeUSD not equal to TVL.

Still need to find pool address

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/dodoex/dodoex-v2',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pairs(first: 1000, orderBy: updatedAt, orderDirection: desc) {
    id
    volumeUSD
    lpFeeRate
    baseToken {
      id
      symbol
      decimals
    }
    quoteToken {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

dodo1 = pd.json_normalize(response1['pairs'],max_level=1)
dodo1

#First 1001-2000 pool
query2 = gql('''
query {
  pairs(first: 1000, skip: 1000, orderBy: updatedAt, orderDirection: desc) {
    id
    volumeUSD
    lpFeeRate
    baseToken {
      id
      symbol
      decimals
    }
    quoteToken {
      id
      symbol
      decimals
    }
  }
}
''')

response2 = client.execute(query2)

dodo2 = pd.json_normalize(response2['pairs'],max_level=1)
dodo2

#First 2001-3000 pool
query3 = gql('''
query {
  pairs(first: 1000, skip: 2000, orderBy: updatedAt, orderDirection: desc) {
    id
    volumeUSD
    lpFeeRate
    baseToken {
      id
      symbol
      decimals
    }
    quoteToken {
      id
      symbol
      decimals
    }
  }
}
''')

response3 = client.execute(query3)

dodo3 = pd.json_normalize(response3['pairs'],max_level=1)
dodo3

#First 3001-4000 pool
query4 = gql('''
query {
  pairs(first: 1000, skip: 3000, orderBy: updatedAt, orderDirection: desc) {
    id
    volumeUSD
    lpFeeRate
    baseToken {
      id
      symbol
      decimals
    }
    quoteToken {
      id
      symbol
      decimals
    }
  }
}
''')

response4 = client.execute(query4)

dodo4 = pd.json_normalize(response4['pairs'],max_level=1)
dodo4

#Combination
dodo_pools = pd.concat([dodo1,dodo2,dodo3,dodo4], ignore_index=True)
dodo_pools.columns = ['pool_address','volumeUSD','lpFeeRate',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["dodo" for x in range(len(dodo_pools.index))])
dodo_pools.insert(loc=0, column='Name', value=Name)
dodo_pools

,Name,pool_address,volumeUSD,lpFeeRate,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,dodo,0xc9f93163c99695c6526b799ebca2207fdf7d61ad,52544585422.4022515048285654400489,0,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6
1,dodo,0x1a97807548ccd5b4671c1576ea70a673859c3a04-0xe...,744.0485783468235686662569092402876,0,0x1a97807548ccd5b4671c1576ea70a673859c3a04,RINU,8,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18
2,dodo,0x8876819535b48b551c9e97ebc07332c7482b4b2d,770841256.7325236175276534372457371,0.003,0x43dfc4159d86f3a37a5a4b3d4580b888ad7d4ddd,DODO,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
3,dodo,0x3007083eaa95497cd6b2b809fb97b6a30bdf53d3-0xe...,68147.2666090364042066482338532987,0,0x3007083eaa95497cd6b2b809fb97b6a30bdf53d3,PSYOP,18,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18
4,dodo,0x3058ef90929cb8180174d74c507176cca6835d73,31347306484.21741377771674151693098,0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6
...,...,...,...,...,...,...,...,...,...,...
3995,dodo,0x18677637bc6e45f38308193deb7de7ec09029e06-0xe...,62.1998922249901956043912036426098,0,0x18677637bc6e45f38308193deb7de7ec09029e06,SHIBL,18,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18
3996,dodo,0xdac17f958d2ee523a2206206994597c13d831ec7-0x5...,500.082233029973595360791002853622,0,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,0x577fee283e776eec29c9e4d258431982780a38a8,PEPA,9
3997,dodo,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48-0x0...,0,0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0x092fa97ec3860dd8f8f6df8c517a026e0225b885,PEEPO,18
3998,dodo,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee-0x6...,565.0773966649522575850997066934218,0,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,ETH,18,0x6d148333ace7040e69b53dde94800cb05ce085d0,POGGY,18


## SmartDex
not include gas fee

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/akashzeromile/smartdex-subgraph',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pairs(first: 1000, orderBy: reserveUSD, orderDirection: desc) {
    id
    reserveUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

smartdex_pool = pd.json_normalize(response1['pairs'],max_level=1)
smartdex_pool

#Combination
smartdex_pool.columns = ['pool_address','tvl',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["smartdex_pool" for x in range(len(smartdex_pool.index))])
smartdex_pool.insert(loc=0, column='Name', value=Name)
smartdex_pool

,Name,pool_address,tvl,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,smartdex_pool,0xdb0194372acecacd58f43d59b8a07b68ff2fe5e2,80066.86149005498129091246985109454,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xd7ecf95cf7ef5256990beaf4ac895cd9e64cb947,pBTC,18
1,smartdex_pool,0x3b8e0fe3c6332c8834b59f63af390ca2e85885d3,4952.425255054417826981143590802277,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,WMATIC,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6
2,smartdex_pool,0xec3ac6f749fe878ebb33bdb545e302d8062b7856,4385.361986040999167319875110761025,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xad684e79ce4b6d464f2ff7c3fd51646892e24b96,NIOX,4
3,smartdex_pool,0xf938d98ade027175271f813b4f042bc8fba769bc,3476.01487754817883348527325525835,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,USDC,6,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,6
4,smartdex_pool,0x0ed159b86f52b4a0c626cd5aed5938800bc82e9e,3148.93213209363780981160564183288,0x9e254a4fb0cc003c8679e48cb1777ed9bf03c5c4,Test,18,0xdd667bf15e34b126eaa53198c207e5390374bc1a,DEV,4
...,...,...,...,...,...,...,...,...,...
80,smartdex_pool,0x36d2c17542b57fabfc816d49e67b42cf60453644,0,0x30a8e1c256143ad440faa9042722929b0bc0fc7d,bat,18,0xb1cb4ac6b1685dcbe691b9246406748805aa6fa8,POLYSAFE,9
81,smartdex_pool,0x3287070c3cf74ab9d2e6ba5d97ca4d8b669ffcfb,0,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,WBTC,8,0x46d3ec8ce3ec767414f16fe12176de23e3e5b46a,SXC,18
82,smartdex_pool,0x30d5ee227231e80a32ce5ac4d8ce0fbde88fa7be,0,0x045a17c2ad1fa49af10f122ab517762538c596ad,NIOXV2,18,0x29df98ec262ac614a7cee6e16ec698902c79ff78,ACDC,18
83,smartdex_pool,0x239db90a3b4855e699fae623a4f0aca193c05879,0,0x30a8e1c256143ad440faa9042722929b0bc0fc7d,bat,18,0x3cef98bb43d732e2f285ee605a8158cde967d219,BAT,18


## DODO New

In [ ]:
transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/dodoex/dodo-uniswap-v3-mainnet',
    verify=True,
    retries=3)

client = Client(
    transport = transport)

#First 1000 pool
query1 = gql('''
query {
  pools(first: 1000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response1 = client.execute(query1)

dodo1 = pd.json_normalize(response1['pools'],max_level=1)

#First 1001-2000 pool
query2 = gql('''
query {
  pools(first: 1000, skip: 1000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response2 = client.execute(query2)

dodo2 = pd.json_normalize(response2['pools'],max_level=1)

#First 2001-3000 pool
query3 = gql('''
query {
  pools(first: 1000, skip: 2000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response3 = client.execute(query3)

dodo3 = pd.json_normalize(response3['pools'],max_level=1)

#First 3001-4000 pool
query4 = gql('''
query {
  pools(first: 1000, skip: 3000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response4 = client.execute(query4)

dodo4 = pd.json_normalize(response4['pools'],max_level=1)

#First 4001-5000 pool
query5 = gql('''
query {
  pools(first: 1000, skip: 4000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response5 = client.execute(query5)

dodo5 = pd.json_normalize(response5['pools'],max_level=1)

#First 5001-6000 pool
query6 = gql('''
query {
  pools(first: 1000, skip: 5000, orderBy: totalValueLockedUSD, orderDirection: desc) {
    id
    totalValueLockedUSD
    token0 {
      id
      symbol
      decimals
    }
    token1 {
      id
      symbol
      decimals
    }
  }
}
''')

response6 = client.execute(query6)

dodo6 = pd.json_normalize(response6['pools'],max_level=1)

#Combination
dodo_pools = pd.concat([dodo1,dodo2,dodo3,dodo4,dodo5,dodo6], ignore_index=True)
dodo_pools.columns = ['pool_address','tvl',
                         'token1','token1_symbol','token1_decimals',
                         'token2','token2_symbol','token2_decimals']
Name = pd.Series(["dodo" for x in range(len(dodo_pools.index))])
dodo_pools.insert(loc=0, column='Name', value=Name)
dodo_pools.insert(loc=3, column='fee', value=0.3)
dodo_pools

,Name,pool_address,tvl,fee,token1,token1_symbol,token1_decimals,token2,token2_symbol,token2_decimals
0,dodo,0x277667eb3e34f134adf870be9550e9f323d0dc24,1110167010695.785900004139211677005,0.3,0x160de4468586b6b2f8a92feb0c260fc6cfc743b1,ease.org,18,0xea5edef1c6ed1be1bcba4617a1c5a994e9018a43,ez-cvxsteCRV,18
1,dodo,0xa850478adaace4c08fc61de44d8cf3b64f359bec,12887625156.60443191907094774602519,0.3,0x12b32f10a499bf40db334efe04226cca00bf2d9b,UMIIE,18,0x5ed60a121159481675bad3e648ba4c89753e056f,UMIIE2,18
2,dodo,0x8c0411f2ad5470a66cb2e9c64536cfb8dcd54d51,1432077483.208630196467118329099649,0.3,0x160de4468586b6b2f8a92feb0c260fc6cfc743b1,ease.org,18,0xea5edef1fcf717327440646e7302ca058dde844f,ez-yvCurve-IronBank,18
3,dodo,0x055284a4ca6532ecc219ac06b577d540c686669d,1322031013.848042060988788389158996,0.3,0x160de4468586b6b2f8a92feb0c260fc6cfc743b1,ease.org,18,0xea5edef10f8255cd03541e8ff163a2836e0d7bd3,ez-SLP-WBTC-WETH,18
4,dodo,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,452759857.4311762923798492296502304,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
...,...,...,...,...,...,...,...,...,...,...
5995,dodo,0x42243093ee2f7a01879cdd0a135e9eeb055892b4,141.576429,0.3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,0xa6787bcba1d1775906a64799a60df7b8011f3f46,KSTND,18
5996,dodo,0xb746bc6e8822437a932e8f8210556980b963515d,141.3055210752594455261885921140007,0.3,0x09a8f29d61f13a8bceeb71266c91eebe7cecd817,FZ,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
5997,dodo,0xeb7de56d3e71111f5f6e969b6a2caa7dd56f79e6,141.2067959197345738007243109519881,0.3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,0xe23d24e26aeb94d3b29cc89e6bd114194dd2f3d7,LATR,18
5998,dodo,0xa4b35daf88a8c894c4664f0625c6e263c61bfcc9,141.1993902015297713210334048696588,0.3,0xb4bd4628e6efb0cb521d9ec35050c75840320374,frETH,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18
